# Modélisation des probas

In [20]:
import os
import pandas as pd
os.getcwd()

path = "c:\\Users\\esthelle.kuissu\\OneDrive - Exiom Partners\\Documents\\Code_rapport_stage\\Bases sinistres"

datap = pd.read_csv(path + "\\bdd_sinistres_MODELPROBA.csv", sep=";", encoding="utf-8")

C:\Users\esthelle.kuissu\AppData\Local\Temp\ipykernel_5256\4172452751.py:7: DtypeWarning: Columns (24,83) have mixed types. Specify dtype option on import or set low_memory=False.
  datap = pd.read_csv(path + "\\bdd_sinistres_MODELPROBA.csv", sep=";", encoding="utf-8")


In [21]:
import pandas as pd

# --- 0. Noms des colonnes montant (adapte si orthographe différente)
mo_col     = 'MT_MO_REPARABLE'
ip_col     = 'MT_IP_REPARABLE'
piece_col  = 'MT_PIECES_REPARABLE'

# --- 1. Fonction qui renvoie la combinaison poste(s) activé(s)
def structure_postes(row):
    postes = []
    if row[mo_col]    > 0: postes.append('MO')
    if row[ip_col]    > 0: postes.append('IP')
    if row[piece_col] > 0: postes.append('PIECES')
    return '+'.join(postes) if postes else 'AUCUN'

datap['structure_sinistre'] = datap.apply(structure_postes, axis=1)

# --- 2. Mappage texte → identifiant numérique (1–7)
map_structure_id = {
    'MO'                 : 1,
    'IP'                 : 2,
    'PIECES'             : 3,
    'MO+IP'              : 4,
    'MO+PIECES'          : 5,
    'IP+PIECES'          : 6,
    'MO+IP+PIECES'       : 7
    # (On ignore le cas "AUCUN" si inexistant : sinon lui donner 0)
}

datap['structure_id'] = datap['structure_sinistre'].map(map_structure_id)

# --- 3. Vérification : table de fréquence
freq = (
    datap.groupby(['structure_id', 'structure_sinistre'])
      .size()
      .reset_index(name='n_sinistres')
      .sort_values('structure_id')
)

print(freq.to_string(index=False))


 structure_id structure_sinistre  n_sinistres
            1                 MO           15
            2                 IP            1
            4              MO+IP        81350
            5          MO+PIECES          356
            6          IP+PIECES           26
            7       MO+IP+PIECES       920097


In [22]:
pieces_seul = datap[
    (datap['MT_PIECES_REPARABLE'] > 0) &
    (datap['MT_MO_REPARABLE'] == 0) &
    (datap['MT_IP_REPARABLE'] == 0)
]
print("Nb de sinistres 'PIECES seule' :", len(pieces_seul))


Nb de sinistres 'PIECES seule' : 0


In [23]:
print(list(datap.columns))

['cle_sin', 'date_evt', 'date_evt_decla', 'clos', 'sans_suite', 'quart_resp', 'date_premsous', 'date_dernrempl', 'formule_ratt', 'cond_indet', 'date_nais', 'age', 'date_permis', 'anc_permis', 'sexe', 'profess', 'sitfam', 'orig_attest', 'aac', 'date_sortie_veh', 'age_veh', 'usage2', 'franch', 'rachat_francBG', 'garag', 'marq', 'modele2', 'generation', 'modele_desc', 'caros', 'segment', 'energ', 'num_sra', 'cylin', 'pdyne', 'pkw', 'pfisc', 'vites', 'pvid', 'poidspuiss', 'sport', 'frein', 'cdval', 'trans', 'alim', 'suspe', 'boite', 'val', 'couple', 'airbag', 'crash', 'co2min', 'elt_secu', 'vu', 'pickup', 'luxe', 'prestige', 'soumis_crm', 'crm', 'boncond', 'nab50', 'bureau_ratt', 'fract_paiem_soc', 'formule_gar', 'petit_rouleur', 'systeme', 'exercice', 'GARANTIE_HOST', 'MT_FRANCHISE', 'MT_CHARGE_BRUTE', 'MT_CHARGE_NETTE', 'MT_REGLEMENT_SIN', 'MT_REGLEMENT_COM', 'MT_ABANDON_RECOURS', 'MT_FRAIS_HONO', 'MT_PROVISIONS', 'MT_RECOURS_TOTAL', 'MT_ESTIMATION_RECOURS_RESTANTS', 'MT_AVANCE_RECOURS',

In [24]:
import pandas as pd

# Création des colonnes indicatrices (uint8 → 0/1)
dummies = pd.get_dummies(datap['exercice'], prefix='year').astype(int)

# (Optionnel) on retire l’année de base 2019 pour éviter la colinéarité
dummies = dummies.drop(columns=['year_2019'])

# Fusion avec ton DataFrame
datap = pd.concat([datap, dummies], axis=1)


In [25]:
import pandas as pd

# 0) ton DataFrame initial ----------------------------------------------------
# df = pd.read_csv("ton_fichier.csv")  # ou déjà présent en mémoire

# 1) liste explicite de TOUTES tes variables (copiée depuis ta question) -----
all_vars = [
    'cle_sin', 'date_evt', 'date_evt_decla', 'clos', 'sans_suite', 'quart_resp',
    'date_premsous', 'date_dernrempl', 'formule_ratt', 'cond_indet', 'date_nais',
    'age', 'date_permis', 'anc_permis', 'sexe', 'profess', 'sitfam', 'orig_attest',
    'aac', 'date_sortie_veh', 'age_veh', 'usage2', 'franch', 'rachat_francBG',
    'garag', 'marq', 'modele2', 'generation', 'modele_desc', 'caros', 'segment',
    'energ', 'num_sra', 'cylin', 'pdyne', 'pkw', 'pfisc', 'vites', 'pvid',
    'poidspuiss', 'sport', 'frein', 'cdval', 'trans', 'alim', 'suspe', 'boite',
    'val', 'couple', 'airbag', 'crash', 'co2min', 'elt_secu', 'vu', 'pickup',
    'luxe', 'prestige', 'soumis_crm', 'crm', 'boncond', 'nab50', 'bureau_ratt',
    'fract_paiem_soc', 'formule_gar', 'petit_rouleur', 'systeme', 'exercice',
    'GARANTIE_HOST', 'MT_FRANCHISE', 'MT_CHARGE_BRUTE', 'MT_CHARGE_NETTE',
    'MT_REGLEMENT_SIN', 'MT_REGLEMENT_COM', 'MT_ABANDON_RECOURS', 'MT_FRAIS_HONO',
    'MT_PROVISIONS', 'MT_RECOURS_TOTAL', 'MT_ESTIMATION_RECOURS_RESTANTS',
    'MT_AVANCE_RECOURS', 'MT_AVANCE_RECOURS_RECOURUS', 'MT_PAIEMENTS',
    'MT_CHARGE_BRUTE_ECO', 'MT_CHARGE_NETTE_ECO', 'ZN_NO_RAPPORT_EXPTIS',
    'MT_VAL_REMPL_VEH_TTC', 'MT_TOTAL_REPARABLE', 'NB_REP', 'MT_MO_REPARABLE',
    'NBHH_MO_REPARABLE', 'TAUX_HORAIRE_MO', 'MT_IP_REPARABLE',
    'NBHH_IP_REPARABLE', 'MT_PIECES_REPARABLE', 'NB_PIECES_REEMPLOI',
    'MT_PIECES_REEMPLOI', 'RE_AVEC_PIECE_REEMPLOI', 'RE_AVEC_PIECE_REPAR_MP',
    'RE_AVEC_PIECE_REMPL_MP', 'MT_REMISE', 'MT_FORFAIT', 'MT_FORFAIT_ERD',
    'LIB_CHOC', 'LIB_INTENS', 'GARAGE_AGREE_O_N', 'DPT_REP', '_RESEAU_',
    'NB_PIECE_REMPLACEE', 'NB_PIECE_REPAREE', 'veh_info_manquante', 'date_ord',
    'date_sortie_veh_imputee', 'annee_sortie_veh', 'date_nais_f',
    'date_permis_f', 'formule_ratt_g', 'sitfam_g', 'usage2_g', 'orig_attest_g',
    'caros_g', 'segment_g', 'energ_g', 'frein_g', 'alim_g', 'suspe_g',
    'crash_g', 'fract_paiem_soc_g', 'cdval_g', 'modele2_g', 'profess_g',
    'marq_g', 'garag_g', 'Region', 'Zone', 'structure_sinistre', 'structure_id',
    'year_2020', 'year_2021', 'year_2022', 'year_2023', 'year_2024'
]

# 2) distinguer variables montants vs non montants ---------------------------
montants     = [v for v in all_vars if v.startswith('MT_')]
non_montants = [v for v in all_vars if v not in montants]

print(f"{len(montants)} variables de montant supprimées.")
print(f"{len(non_montants)} variables conservées.")

# 3) créer la base sans montants ---------------------------------------------
datap_sm = datap[non_montants].copy()

# 4) enregistrer si besoin ----------------------------------------------------
#datap_sm.to_csv("baseproba_sans_montants.csv", index=False, encoding="utf-8")


24 variables de montant supprimées.
116 variables conservées.


In [26]:
datap_sm.head()

,cle_sin,date_evt,date_evt_decla,clos,sans_suite,quart_resp,date_premsous,date_dernrempl,formule_ratt,cond_indet,...,garag_g,Region,Zone,structure_sinistre,structure_id,year_2020,year_2021,year_2022,year_2023,year_2024
0,191840439C A 018,02/01/2019,04/01/2019,O,N,plus_50pc,29/06/1987,09/01/2015,PRO,N,...,Petit garage,Nouvelle-Aquitaine,Ouest,MO+IP+PIECES,7,0,0,0,0,0
1,191840048C A 001,01/01/2019,02/01/2019,N,N,plus_50pc,10/11/2018,2023-12-31 00:00:00,PRO,N,...,Garage moyen,Pays de la Loire,Ouest,MO+IP+PIECES,7,0,0,0,0,0
2,191840501C A 010,03/01/2019,04/01/2019,O,N,plus_50pc,13/07/2016,03/04/2018,PRO,N,...,Garage moyen,Pays de la Loire,Ouest,MO+IP+PIECES,7,0,0,0,0,0
3,191840759C A 001,06/01/2019,07/01/2019,O,N,plus_50pc,25/10/2015,2023-12-31 00:00:00,PRO,N,...,Garage moyen,Pays de la Loire,Ouest,MO+IP+PIECES,7,0,0,0,0,0
4,191840075AV A 003,01/01/2019,02/01/2019,O,N,plus_50pc,10/06/2015,11/03/2016,PRO,N,...,Garage moyen,NaN,I,MO+IP+PIECES,7,0,0,0,0,0


In [27]:
print(list(datap_sm.columns))

['cle_sin', 'date_evt', 'date_evt_decla', 'clos', 'sans_suite', 'quart_resp', 'date_premsous', 'date_dernrempl', 'formule_ratt', 'cond_indet', 'date_nais', 'age', 'date_permis', 'anc_permis', 'sexe', 'profess', 'sitfam', 'orig_attest', 'aac', 'date_sortie_veh', 'age_veh', 'usage2', 'franch', 'rachat_francBG', 'garag', 'marq', 'modele2', 'generation', 'modele_desc', 'caros', 'segment', 'energ', 'num_sra', 'cylin', 'pdyne', 'pkw', 'pfisc', 'vites', 'pvid', 'poidspuiss', 'sport', 'frein', 'cdval', 'trans', 'alim', 'suspe', 'boite', 'val', 'couple', 'airbag', 'crash', 'co2min', 'elt_secu', 'vu', 'pickup', 'luxe', 'prestige', 'soumis_crm', 'crm', 'boncond', 'nab50', 'bureau_ratt', 'fract_paiem_soc', 'formule_gar', 'petit_rouleur', 'systeme', 'exercice', 'GARANTIE_HOST', 'ZN_NO_RAPPORT_EXPTIS', 'NB_REP', 'NBHH_MO_REPARABLE', 'TAUX_HORAIRE_MO', 'NBHH_IP_REPARABLE', 'NB_PIECES_REEMPLOI', 'RE_AVEC_PIECE_REEMPLOI', 'RE_AVEC_PIECE_REPAR_MP', 'RE_AVEC_PIECE_REMPL_MP', 'LIB_CHOC', 'LIB_INTENS', 'GA

In [28]:
# ---------------------------------------------
# 1. Liste des variables à exclure
# ---------------------------------------------
exclure = [
    'date_evt_decla', 'clos', 'sans_suite', 'quart_resp', 'date_premsous',
    'date_dernrempl', 'formule_ratt', 'cond_indet', 'date_nais', 'Zone',
    'date_permis', 'anc_permis', 'sexe', 'profess', 'sitfam', 'orig_attest',
    'date_sortie_veh', 'usage2', 'franch', 'rachat_francBG',
    'garag', 'marq', 'modele2', 'generation', 'modele_desc', 'caros',
    'segment', 'energ', 'num_sra', 'cylin', 'pdyne', 'pfisc', 'vites',
    'pvid', 'bureau_ratt', 'fract_paiem_soc', 'formule_gar', 'petit_rouleur',
    'systeme', 'exercice', 'GARANTIE_HOST', 'ZN_NO_RAPPORT_EXPTIS', 'NB_REP',
    'NBHH_MO_REPARABLE', 'TAUX_HORAIRE_MO', 'NBHH_IP_REPARABLE',
    'NB_PIECES_REEMPLOI', 'RE_AVEC_PIECE_REEMPLOI', 'RE_AVEC_PIECE_REPAR_MP',
    'RE_AVEC_PIECE_REMPL_MP', 'veh_info_manquante', 'date_ord',
    'date_sortie_veh_imputee'
]

# ---------------------------------------------
# 2. Variable d’origine : par ex. non_montants
#    (remplace par le nom réel de ta liste)
# ---------------------------------------------
# non_montants = [...]  # ta liste existante

# ---------------------------------------------
# 3. Création de la nouvelle liste filtrée
# ---------------------------------------------
vars_filtrees = [col for col in non_montants if col not in exclure]

print(f"{len(non_montants)} variables initiales → {len(vars_filtrees)} après exclusion.")

datap_s = datap_sm[vars_filtrees].copy()


116 variables initiales → 63 après exclusion.


In [29]:
print(list(datap_s.columns))

['cle_sin', 'date_evt', 'age', 'aac', 'age_veh', 'pkw', 'poidspuiss', 'sport', 'frein', 'cdval', 'trans', 'alim', 'suspe', 'boite', 'val', 'couple', 'airbag', 'crash', 'co2min', 'elt_secu', 'vu', 'pickup', 'luxe', 'prestige', 'soumis_crm', 'crm', 'boncond', 'nab50', 'LIB_CHOC', 'LIB_INTENS', 'GARAGE_AGREE_O_N', 'DPT_REP', '_RESEAU_', 'NB_PIECE_REMPLACEE', 'NB_PIECE_REPAREE', 'annee_sortie_veh', 'date_nais_f', 'date_permis_f', 'formule_ratt_g', 'sitfam_g', 'usage2_g', 'orig_attest_g', 'caros_g', 'segment_g', 'energ_g', 'frein_g', 'alim_g', 'suspe_g', 'crash_g', 'fract_paiem_soc_g', 'cdval_g', 'modele2_g', 'profess_g', 'marq_g', 'garag_g', 'Region', 'structure_sinistre', 'structure_id', 'year_2020', 'year_2021', 'year_2022', 'year_2023', 'year_2024']


In [31]:
# 4) enregistrer si besoin ----------------------------------------------------
datap_s.to_csv(path + "basefinproba_sans_montants.csv", index=False, encoding="utf-8")



In [32]:
datap_s.head()

,cle_sin,date_evt,age,aac,age_veh,pkw,poidspuiss,sport,frein,cdval,...,marq_g,garag_g,Region,structure_sinistre,structure_id,year_2020,year_2021,year_2022,year_2023,year_2024
0,191840439C A 018,02/01/2019,84.0,N,3.0,66.0,12.0,nn_spt,ABS,V06,...,Généralistes,Petit garage,Nouvelle-Aquitaine,MO+IP+PIECES,7,0,0,0,0,0
1,191840048C A 001,01/01/2019,18.0,O,6.0,51.5,15.0,nn_spt,ABS,V06,...,Autres,Garage moyen,Pays de la Loire,MO+IP+PIECES,7,0,0,0,0,0
2,191840501C A 010,03/01/2019,67.0,N,6.0,180.0,7.0,spt,ABS,V24,...,Premium,Garage moyen,Pays de la Loire,MO+IP+PIECES,7,0,0,0,0,0
3,191840759C A 001,06/01/2019,40.0,N,12.0,78.0,13.0,nn_spt,ABS,V09,...,Généralistes,Garage moyen,Pays de la Loire,MO+IP+PIECES,7,0,0,0,0,0
4,191840075AV A 003,01/01/2019,47.0,N,3.0,84.0,12.0,nn_spt,ABS,V11,...,Généralistes,Garage moyen,NaN,MO+IP+PIECES,7,0,0,0,0,0


In [36]:
print(len(list(datap_s.columns)))

63


## Petite sélection de variables

In [40]:
from sklearn.preprocessing import OneHotEncoder

# Remplace 'sparse' par 'sparse_output'
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [35]:
import pandas as pd
import numpy as np

to_exclude = set()

# a) identifiants, cible
to_exclude.update(['cle_sin', 'structure_id', 'structure_sinistre'])


# c) éventuellement d’autres colonnes (dates si tu ne les transformes pas)
to_exclude.update(['date_evt', 'date_premsous'])

# ------------------------------------------------------------------
# 1) Détection auto : quali vs quanti
# ------------------------------------------------------------------
# numériques : int, float, bool
quant_vars = [
    c for c in datap_s.columns
    if (datap_s[c].dtype.kind in 'iufcb')    # integer, unsigned, float, complex, bool
    and c not in to_exclude
]

# qualitatives : object ou category
qual_vars = [
    c for c in datap_s.columns
    if (datap_s[c].dtype == 'object' or str(datap_s[c].dtype) == 'category')
    and c not in to_exclude
]

print(f"{len(quant_vars)} variables numériques retenues")
print(f"{len(qual_vars)} variables qualitatives retenues")


20 variables numériques retenues
39 variables qualitatives retenues


In [43]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


# -----------------------------------------------------------------------------
# 2. Encodage one-hot (drop_first pour éviter colinéarité parfaite)
# -----------------------------------------------------------------------------
from sklearn.preprocessing import OneHotEncoder

# 1. Encodage OneHot avec gestion des modalités inconnues
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

X_ohe = pd.DataFrame(
    ohe.fit_transform(datap_s[qual_vars]),
    columns=ohe.get_feature_names_out(qual_vars),
    index=datap_s.index
)

# 2. Concatène variables numériques + variables encodées
X = pd.concat([datap_s[quant_vars].reset_index(drop=True),
               X_ohe.reset_index(drop=True)],
              axis=1)

# 3. Supprime les colonnes constantes (nécessaire pour le VIF)
constantes = X.columns[X.nunique() <= 1]
X = X.drop(columns=constantes)

print(f"Shape finale de X : {X.shape}")



MemoryError: Unable to allocate 424. GiB for an array with shape (1001845, 56766) and data type float64

In [42]:
import sklearn
print(sklearn.__version__)

1.7.0


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["variables"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
